**(SRGAN) with pytorch**

 we build a model that can realistically increase image resolution. 


Using *PyTorch 1.4* in *Python 3.6*.

In [17]:
#this code to connect or mount your Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys

sys.path.append('/content/drive/My Drive/Aphrodite-SRGAN')

**our checkpoints are available**    [SRGAN (TAR)]( https://drive.google.com/open?id=1ePooVQcEbIjEZfE2ED1dmtCVj-xUbg1c )
you can download it .
we train about 3000 epoch from [DIV2K](https://data.vision.ee.ethz.ch/cvl/DIV2K/) dataset .
if you want to train another dataset this [link](https://drive.google.com/open?id=1qEEX29LyVP2NjNxYw2WR-KcehjdSCjEM) for our project you can `run` file : train-srgan.py

In [0]:


   
    
#----------------------------------------------------------
from utils import *
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from datasets import SRDataset
from PIL import Image
import matplotlib.pyplot as plt
from torch import argmax
from google.colab import files
from PIL import Image, ImageDraw, ImageFont
from PIL import Image, ImageEnhance  , ImageFilter
from google.colab import files
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def afro(url):
  
# Data

  test_data_names = ["DIV2k"]

# Model checkpoints
  srgan_checkpoint = "/content/drive/My Drive/SRGAN (TAR)/checkpoint_srgan.pth.tar"
  
# Load model,  the   the SRGAN

  srgan_generator = torch.load(srgan_checkpoint)['generator'].to(device)
  srgan_generator.eval()
  model = srgan_generator

# Evaluate
  for test_data_name in test_data_names:
  

    # Custom dataloader
     test_dataset = SRDataset(url= url,
                             split='test',
                             crop_size=0,
                             scaling_factor=4,
                             lr_img_type='imagenet-norm',
                             hr_img_type='[-1, 1]',
                             test_data_name=test_data_name)
     test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4,
                                              pin_memory=True)

    

    # Prohibit gradient computation explicitly because I had some problems with memory
     with torch.no_grad():
        # Batches
         for i, (lr_imgs, hr_imgs) in enumerate(test_loader):
            # Move to default device
            lr_imgs = lr_imgs.to(device)  # (batch_size (1), 3, w / 4, h / 4), imagenet-normed
            hr_imgs = hr_imgs.to(device)  # (batch_size (1), 3, w, h), in [-1, 1]
            # Forward prop.
            sr_imgs = model(lr_imgs)# (1, 3, w, h), in [-1, 1]

           
           
            sr_img_srgan = sr_imgs.squeeze(0).cpu().detach()
            sr_img_srgan = convert_image(sr_img_srgan, source='[-1, 1]', target='pil')
           
            mr_img = sr_img_srgan.resize((int(sr_img_srgan.width *1), int(sr_img_srgan.height *1)))
            smoothenedImage = mr_img.filter(ImageFilter.SMOOTH)

            moreSmoothenedImage = smoothenedImage.filter(ImageFilter.SMOOTH_MORE)
            im3 = ImageEnhance.Sharpness(moreSmoothenedImage) 
            
            print(mr_img.width)
            # showing resultant image 
            
            plt.imshow(im3.enhance(-5.0))
           # plt.imshow(sr_img_srgan)
            # Calculate PSNR and SSIM
            sr_imgs_y = convert_image(sr_imgs, source='[-1, 1]', target='y-channel').squeeze(
                0)  # (w, h), in y-channel
            hr_imgs_y = convert_image(hr_imgs, source='[-1, 1]', target='y-channel').squeeze(0)  # (w, h), in y-channel
            
    


  print("\n")
#------------------------------------------------------------------------

afro('/content/drive/My Drive/Aphrodite-Project/images/low1.png')
    

you can test the code : just change the *url* in `afro(url)`